# Libraries

In [1]:
%pip install llama-index -q
%pip install transformers -q
%pip install torch -q
%pip install llama-index-llms-groq -q
%pip install sentence-transformers -q
%pip install "llama-index-embeddings-huggingface" -q
%pip install kdbai-client -q
%pip install llama-index-vector-stores-kdbai -q
%pip install kdbai_client -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 

In [13]:
import pandas as pd
from typing import List, Dict
from llama_index.core import VectorStoreIndex, ServiceContext, Document
from llama_index.core.node_parser import SentenceSplitter, MarkdownNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage
import kdbai_client as kdbai

import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Data Loading

In [27]:
def load_data(csv_path: str, text_col: List[str], metadata_cols: List[str]) -> List[Document]:
  """
  Load documents and include class in metadata
  """
  df = pd.read_csv(csv_path)
  df.fillna("nan", inplace=True)
  documents = []
  cols = ['original_doc_id', 'class', 'issuing_authority', 'title', 'issue_date', 'reference_number']
  for _, row in df.iterrows():
      text = str(row[text_col])
      doc = Document(
          text=text,
          metadata= {cols[i]: row[col] for i, col in enumerate(metadata_cols)}
      )
      documents.append(doc)
  return documents

DATA_PATH = "/content/drive/MyDrive/Omdena/Regulatory RAG (SL Chapter)/code/model dev/data/2024_11_28 v0_LK_tea_dataset.csv"
text_col = 'markdown_content'
metadata_cols = ['id', 'class', 'issuing_authority', 'llama_title', 'llama_issue_date', 'llama_reference_number']

all_documents = load_data(DATA_PATH, text_col, metadata_cols)
len(all_documents)

167

In [28]:
set([doc.metadata['class'] for doc in all_documents])

{'circular', 'guideline', 'regulatory'}

In [29]:
circulars_docs = [doc for doc in all_documents if doc.metadata['class'] == 'circular']
len(circulars_docs)

107

In [30]:
guideline_docs = [doc for doc in all_documents if doc.metadata['class'] == 'guideline']
len(guideline_docs)

13

In [31]:
regulatory_docs = [doc for doc in all_documents if doc.metadata['class'] == 'regulatory']
len(regulatory_docs)

47

# Embedding Model

In [20]:
def setup_embedding_model():
    """
    Setup HuggingFace embedding model
    """
    model_name = 'BAAI/bge-small-en-v1.5'
    return HuggingFaceEmbedding(
        model_name=model_name,
        trust_remote_code=True,
        cache_folder="/content/drive/MyDrive/Omdena/Regulatory RAG (SL Chapter)/code/model dev/cached_models/"
        )

embed_model = setup_embedding_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Groq + KDBAI API Setup

In [21]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [22]:
def setup_groq_llm():
    """
    Setup Groq LLM
    """
    groq_api_key = GROQ_API_KEY
    if not groq_api_key:
        raise ValueError("Please set GROQ_API_KEY environment variable")

    return Groq(
        api_key=groq_api_key,
        model="llama3-groq-70b-8192-tool-use-preview",
        temperature=0.0
    )

llm = setup_groq_llm()

# KDBAI API + Session Setup

In [23]:
KDBAI_API_KEY = userdata.get('KDBAI_API_KEY')
KDBAI_SESSION_ENDPOINT = userdata.get('KDBAI_SESSION_ENDPOINT')

In [24]:
def setup_kdbai_api():
  """
  Setup KDBAI Session Endpoint and API
  """

  kdbai_endpoint = KDBAI_SESSION_ENDPOINT
  if not kdbai_endpoint:
        raise ValueError("Please set KDBAI_SESSION_ENDPOINT environment variable")

  kdbai_api_key = KDBAI_API_KEY
  if not kdbai_api_key:
        raise ValueError("Please set KDBAI_API_KEY environment variable")

  return kdbai.Session(
    endpoint=f"https://cloud.kdb.ai/instance/{kdbai_endpoint}",
    api_key=f"{kdbai_api_key}"
    )

session = setup_kdbai_api()

# KDBAI Vector Store Setup

## Session Database

In [45]:
session.databases()

[KDBAI database "default",
 KDBAI database "srilanka_rag_database",
 KDBAI database "srilanka_tri_circulars"]

In [46]:
# ensure no database called "srilanka_tea" exists
try:
    session.database("srilanka_rag_database").drop()
except kdbai.KDBAIException:
    pass

# Create the database
db = session.create_database("srilanka_rag_database")
session.databases()

[KDBAI database "default",
 KDBAI database "srilanka_rag_database",
 KDBAI database "srilanka_tri_circulars"]

## Table Schema + Creation

In [47]:
# List all of the tables in the db
db.tables

[]

In [48]:
# Table - name & schema
circular_table_name = "circular_baseline"
regulation_table_name = "regulation_baseline"
guideline_table_name = "guideline_baseline"

# Same table schema and table index for each class
table_schema = [
        dict(name="document_id", type="bytes"),
        dict(name="text", type="bytes"),
        dict(name="embeddings", type="float32s"),
        dict(name="issue_date", type="str"),
        dict(name="issuing_authority", type="str"),
        dict(name="class", type="str"),
        dict(name="reference_number", type="str"),
        dict(name="title", type="str"),
        dict(name="original_doc_id", type="int16"),
    ]

indexFlat = {
        "name": "flat_index",
        "type": "flat",
        "column": "embeddings",
        "params": {'dims': 384, 'metric': 'CS'} # For similarity metric, choose from Euclidean Distance (L2), Dot Product (IP), or Cosine Similarity (CS).
    }

In [49]:
# First ensure the tables does not already exist
try:
    db.table(circular_table_name).drop()
    db.table(regulation_table_name).drop()
    db.table(guideline_table_name).drop()
except kdbai.KDBAIException:
    pass

# Create tables
circular_table = db.create_table(circular_table_name, table_schema, indexes=[indexFlat])
regulation_table = db.create_table(regulation_table_name, table_schema, indexes=[indexFlat])
guideline_table = db.create_table(guideline_table_name, table_schema, indexes=[indexFlat])

db.tables

[KDBAI table "circular_baseline",
 KDBAI table "regulation_baseline",
 KDBAI table "guideline_baseline"]

In [50]:
circular_table.indexes

[{'name': 'flat_index',
  'type': 'flat',
  'column': 'embeddings',
  'params': {'metric': 'CS', 'dims': 384}}]

## Insert Data into Tables

In [51]:
from llama_index.vector_stores.kdbai import KDBAIVectorStore
from llama_index.core import StorageContext, Settings
from llama_index.core.indices import VectorStoreIndex

In [52]:
Settings.llm = llm
Settings.embed_model = embed_model

In [53]:
%%time

# Vector Store
guideline_vector_store = KDBAIVectorStore(table=guideline_table, index_name="guideline_baseline_index")
regulation_vector_store = KDBAIVectorStore(table=regulation_table, index_name="regulation_baseline_index")
circular_vector_store = KDBAIVectorStore(table=circular_table, index_name="circular_baseline_index")

guideline_storage_context = StorageContext.from_defaults(vector_store=guideline_vector_store)
regulation_storage_context = StorageContext.from_defaults(vector_store=regulation_vector_store)
circular_storage_context = StorageContext.from_defaults(vector_store=circular_vector_store)

CPU times: user 489 µs, sys: 0 ns, total: 489 µs
Wall time: 495 µs


In [54]:
%%time

# Create guideline index
guideline_index = VectorStoreIndex.from_documents(
    guideline_docs,
    storage_context=guideline_storage_context,
    transformations=[MarkdownNodeParser()]
)

CPU times: user 1min 11s, sys: 2.99 s, total: 1min 14s
Wall time: 1min 18s


In [55]:
guideline_table.query()

,document_id,text,embeddings,issue_date,issuing_authority,class,reference_number,title,original_doc_id
0,b'f8360d98-8c5d-49c8-80d3-c30cd5ea2b6b',b'# TEA RESEARCH INSTITUTE OF SRI LANKA\r\n\r\...,"[-0.0362014, -0.03285268, 0.01507808, 0.025595...",nan,Tea Research Institute,guideline,nan,nan,83
1,b'3f576202-8dba-41b5-92ed-b4f41d0e9a46',"b'# GUIDELINES ON THE USE OF ""SOIL QUALITY IND...","[-0.045181602, -0.037191708, 0.03183228, 0.054...",nan,Tea Research Institute,guideline,nan,nan,83
2,b'0c2cb2b1-f055-4033-bf6e-297be420aade',b'# Introduction\r\n\r\nSoil rehabilitation by...,"[-0.026363228, -0.032797795, 0.069582395, 0.06...",nan,Tea Research Institute,guideline,nan,nan,83
3,b'67cc15cc-0baf-4d0c-9742-275da98613a9',"b'# Soil Quality Index\r\n\r\nDevelopment, val...","[-0.009502379, -0.05923057, 0.0214256, 0.03497...",nan,Tea Research Institute,guideline,nan,nan,83
4,b'9ebb4b22-1d5c-4a8e-a57f-5c2cd9767c58',b'# Table 1: Soil parameter values and the sco...,"[-0.034701668, -0.01693064, 0.011039094, 0.028...",nan,Tea Research Institute,guideline,nan,nan,83
...,...,...,...,...,...,...,...,...,...
164,b'cbed007c-10c6-41e7-bc55-67f058cb9533',b'# 3.5\r\n\r\nImplement GMPs (Good Manufactur...,"[-0.037515055, 0.015980398, 0.023546645, -0.01...",December 2021,Tea Research Institute,guideline,04/2021,Interim Guidelines on Strategic Cost Component...,95
165,b'f1a5d29d-7515-4d40-b8b5-b5dbb9a8ad30',b'# 3.6\r\n\r\nSuitably re-siting machinery an...,"[-0.018048534, -0.04113292, 0.036271565, 0.025...",December 2021,Tea Research Institute,guideline,04/2021,Interim Guidelines on Strategic Cost Component...,95
166,b'a30a4f1f-4521-4238-bf8b-66feaf96f78e',b'# 3.7\r\n\r\nAvoid operating the factory und...,"[-0.010552307, -0.0375594, 0.03401511, 0.04288...",December 2021,Tea Research Institute,guideline,04/2021,Interim Guidelines on Strategic Cost Component...,95
167,b'ba52a08e-c100-4d70-bc5f-f42aa1150945',b'# 3.8\r\n\r\nTrain factory staff and workers...,"[-0.023665305, -0.025489056, 0.03445994, 0.031...",December 2021,Tea Research Institute,guideline,04/2021,Interim Guidelines on Strategic Cost Component...,95


In [56]:
%%time

# Create regulations index
regulation_index = VectorStoreIndex.from_documents(
    regulatory_docs,
    storage_context=regulation_storage_context,
    transformations=[MarkdownNodeParser()]
)

CPU times: user 8min 49s, sys: 14.2 s, total: 9min 4s
Wall time: 9min 28s


In [57]:
regulation_table.query()

,document_id,text,embeddings,issue_date,issuing_authority,class,reference_number,title,original_doc_id
0,b'ba92150c-194e-4904-bd04-4bebf61e6aa0',"b'# Tea Subsidy\r\n\r\nActs Nos. 12 of 1958, 6...","[-0.048545588, -0.0344346, -0.009042649, -0.01...","19th September, 1958",Parliament of Sri Lanka,regulatory,"12 of 1958, 66 of 1961, 33 of 1966","Tea Subsidy Act, No. 12 of 1958",0
1,b'8d563f70-ac8b-477c-a686-8df268b0e880',b'# AN ACT TO PROVIDE FOR THE ESTABLISHMENT OF...,"[-0.041892175, -0.048261214, -0.011618911, -0....","19th September, 1958",Parliament of Sri Lanka,regulatory,"12 of 1958, 66 of 1961, 33 of 1966","Tea Subsidy Act, No. 12 of 1958",0
2,b'96c73e15-ccf7-449c-87bc-60e0ef0d8f32',b'# Short title and date of operation.\r\n\r\n...,"[-0.049599204, -0.03894329, -0.00876235, -0.00...","19th September, 1958",Parliament of Sri Lanka,regulatory,"12 of 1958, 66 of 1961, 33 of 1966","Tea Subsidy Act, No. 12 of 1958",0
3,b'1d211520-89cf-4f77-b261-7fc91655d4b0',"b'# Tea Subsidy Fund. [\xc3\x82\xc2\xa7 3, 66 ...","[-0.029680591, -0.057338122, -0.030874945, -0....","19th September, 1958",Parliament of Sri Lanka,regulatory,"12 of 1958, 66 of 1961, 33 of 1966","Tea Subsidy Act, No. 12 of 1958",0
4,b'9ec19dac-3346-4987-a914-9f88090cb3cf',b'# Payments into and out of the Fund. [\xc3\x...,"[-0.031160707, -0.0675217, -0.027405214, -0.01...","19th September, 1958",Parliament of Sri Lanka,regulatory,"12 of 1958, 66 of 1961, 33 of 1966","Tea Subsidy Act, No. 12 of 1958",0
...,...,...,...,...,...,...,...,...,...
1119,b'3d140323-2ca5-4e1f-ab07-1bee831514f8',b'# Notification\r\n\r\nWages Board for the te...,"[-0.0730936, -0.03271118, -0.017699525, -0.019...",13.08.2024,Government of Sri Lanka,regulatory,2397/27,GAZETTE EXTRAORDINARY OF THE DEMOCRATIC SOCIAL...,113
1120,b'0267c654-509c-46f1-936e-5ef1a1de4778',b'# Schedule\r\n\r\nThe Part II of the Wages B...,"[-0.040807415, -0.047702584, -0.009836054, -0....",13.08.2024,Government of Sri Lanka,regulatory,2397/27,GAZETTE EXTRAORDINARY OF THE DEMOCRATIC SOCIAL...,113
1121,b'e6eee53d-f25c-44ed-a446-eeae706c0ca1',b'# Part I : Sec. (I) - GAZETTE EXTRAORDINARY ...,"[-0.043230187, -0.028875167, 0.011248557, -0.0...",13.08.2024,Government of Sri Lanka,regulatory,2397/27,GAZETTE EXTRAORDINARY OF THE DEMOCRATIC SOCIAL...,113
1122,b'1f3cf7db-94e5-4022-84d6-bbc51ef7ea7f',b'# Part II',"[-0.040043954, -0.023455815, 0.01238047, -0.06...",13.08.2024,Government of Sri Lanka,regulatory,2397/27,GAZETTE EXTRAORDINARY OF THE DEMOCRATIC SOCIAL...,113


In [58]:
%%time

# Create circular index
circular_index = VectorStoreIndex.from_documents(
    circulars_docs,
    storage_context=circular_storage_context,
    transformations=[MarkdownNodeParser()]
)

CPU times: user 8min 22s, sys: 10.9 s, total: 8min 33s
Wall time: 9min 12s


In [59]:
circular_table.query()

,document_id,text,embeddings,issue_date,issuing_authority,class,reference_number,title,original_doc_id
0,b'4b2a50ed-9a6c-4a2d-9ebf-e81f06f38559',b'# ANALYTICAL LABORATORY OF SRILANKA TEA BOARD',"[0.020139394, -0.031413738, 0.034866665, 0.003...",24.06.2013,Tea Board Analytical Lab,circular,AL/GI,GENERAL INSTRUCTION FOR THE SUBMISSION OF TEA ...,29
1,b'fa9b0c62-9a3e-4d81-85a9-905d479b739b',b'# GENERAL INSTRUCTION FOR THE SUBMISSION OF ...,"[0.012524138, -0.025750542, 0.037893508, 0.016...",24.06.2013,Tea Board Analytical Lab,circular,AL/GI,GENERAL INSTRUCTION FOR THE SUBMISSION OF TEA ...,29
2,b'4297d3ae-15ec-40cd-9cec-c7e0afae9575',b'# 1. Submission of tea sample\r\n\r\n- Sampl...,"[-0.003938523, -0.002252292, 0.0009754361, -0....",24.06.2013,Tea Board Analytical Lab,circular,AL/GI,GENERAL INSTRUCTION FOR THE SUBMISSION OF TEA ...,29
3,b'4b9e0942-4d93-42ce-877b-736b1c654890',b'# 2. Sample Container and size of the tea sa...,"[0.015268284, -0.030547412, -0.0004954031, -0....",24.06.2013,Tea Board Analytical Lab,circular,AL/GI,GENERAL INSTRUCTION FOR THE SUBMISSION OF TEA ...,29
4,b'0075e2a1-c20f-46ac-b470-85166b287e0a',b'# 3. Labeling of tea sample\r\n\r\n- An iden...,"[0.021693101, -0.013306763, -0.003444087, -0.0...",24.06.2013,Tea Board Analytical Lab,circular,AL/GI,GENERAL INSTRUCTION FOR THE SUBMISSION OF TEA ...,29
...,...,...,...,...,...,...,...,...,...
1225,b'bddd95d5-6473-456c-a97a-d60ead396eb4',b'# Category of packing/Net Weight\r\n\r\n|210...,"[-0.02512031, -0.031006545, 0.025886627, -0.01...",02nd July 2010,Tea Board,circular,Circular No TCIEXIHS,EXPORTATION OF TEA PREPARATION OF CUSTOMS GOOD...,169
1226,b'43f3ad24-9aa6-4700-8bb5-f237283a8478',b'# Cage (Box) 31 of CUSDEC Lor IL',"[-0.002425255, -0.037568837, 0.03928824, 0.022...",02nd July 2010,Tea Board,circular,Circular No TCIEXIHS,EXPORTATION OF TEA PREPARATION OF CUSTOMS GOOD...,169
1227,b'9d58e469-fc50-4366-bc39-35358d379ba6',b'# PACKAGE AND DESCRIPTION QEGCQDS\r\n\r\nMar...,"[-0.021761937, -0.019291947, 0.021511586, 0.03...",02nd July 2010,Tea Board,circular,Circular No TCIEXIHS,EXPORTATION OF TEA PREPARATION OF CUSTOMS GOOD...,169
1228,b'd6670ea9-9310-418b-9b23-bc814a7aabe9',b'# Cage (Box) 33 of CUSDEC Lor I - Commodity ...,"[-0.0065225135, -0.028387783, 0.016303547, 0.0...",02nd July 2010,Tea Board,circular,Circular No TCIEXIHS,EXPORTATION OF TEA PREPARATION OF CUSTOMS GOOD...,169


## Setting up Query Engine

In [60]:
%%time

# Set topK
K = 5

guideline_query_engine = guideline_index.as_query_engine(
    similarity_top_k=K,
    llm=llm,
    vector_store_kwargs={"index": "flat_index"}
    )

regulation_query_engine = regulation_index.as_query_engine(
    similarity_top_k=K,
    llm=llm,
    vector_store_kwargs={"index": "flat_index"}
    )

circular_query_engine = circular_index.as_query_engine(
    similarity_top_k=K,
    llm=llm,
    vector_store_kwargs={"index": "flat_index"}
    )

CPU times: user 234 ms, sys: 29 µs, total: 234 ms
Wall time: 275 ms


## Settup up Query Router Engine

In [64]:
from llama_index.core.tools import QueryEngineTool

guideline_tool = QueryEngineTool.from_defaults(
    query_engine=guideline_query_engine,
    name  = "guidelines_tool",
    description=(
        "Provide information about Sri Lanka Guidelines documents."
    ),
)

regulation_tool = QueryEngineTool.from_defaults(
    query_engine=regulation_query_engine,
    name = "regulations_tool",
    description=(
        "Provide information about Sri Lanka Regulations documents and Acts."
    ),
)

circular_tool = QueryEngineTool.from_defaults(
    query_engine=circular_query_engine,
    name = "circulars_tool",
    description=(
        "Provide information about Sri Lanka Circular documents."
    ),
)

In [74]:
from llama_index.core.query_engine import RouterQueryEngine
# from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector

query_engine = RouterQueryEngine(
    selector=LLMMultiSelector.from_defaults(),
    query_engine_tools=[
        guideline_tool,
        regulation_tool,
        circular_tool
    ],
)

## Querying Vector Store with Questions

In [69]:
from IPython.display import display, Markdown

In [75]:
%%time

input_query = "What are the basic requirements needed to be fulfilled to be a Tea Exporter?"

result = query_engine.query(input_query)
display(Markdown(result.response))

To be a tea exporter, the basic requirements include being registered as an exporter of tea, having an adequate organization, sufficient financial resources, and suitable premises for engaging in the business of an exporter of tea. Additionally, the exporter must not have contravened any provision of the Tea (Tax and Control of Export) Act, No. 16 of 1959.

CPU times: user 105 ms, sys: 2.93 ms, total: 108 ms
Wall time: 1.15 s


In [76]:
%%time

input_query = "What is the leavy amount that exporters should pay for each Kilo of tea being exported?"

result = query_engine.query(input_query)
display(Markdown(result.response))

The leavy amount that exporters should pay for each kilo of tea being exported is not explicitly mentioned in the provided context information. However, the export duty on tea imposed under the Tea Subsidy Act, No. 12 of 1958, is four cents for every pound of tea exported from Ceylon. This export duty is in addition to any export duty on tea levied under any other written law.

CPU times: user 174 ms, sys: 3.94 ms, total: 178 ms
Wall time: 1.72 s


In [77]:
%%time

input_query = "What are the 5 technical topics that are within the TRI circulars?"

result = query_engine.query(input_query)
display(Markdown(result.response))

The 5 technical topics that are within the TRI circulars are:

1. Soil Analysis
2. Arch
3. Cultivation of Tea Soils Forking
4. Advisory Circular No. SP THL 925
5. Advisory Circular No. PA

CPU times: user 127 ms, sys: 1.12 ms, total: 128 ms
Wall time: 996 ms


In [81]:
%%time

input_query = "In terms of tea farming, what is medium shade?"

result = query_engine.query(input_query)
display(Markdown(result.response))

In terms of tea farming, medium shade refers to a specific level of shade that is considered optimal for tea plant growth and productivity. It is a balance between full shade and full sun, allowing for adequate sunlight penetration while also providing protection from extreme weather conditions. This level of shade helps to promote healthy plant growth, improve yield, and enhance the quality of the tea leaves.

CPU times: user 101 ms, sys: 1.07 ms, total: 103 ms
Wall time: 1.22 s


In [80]:
%%time

input_query = "What are the duties of a regulatory officer with respect to medium shade in tea farming?"

result = query_engine.query(input_query)
display(Markdown(result.response))

The duties of a regulatory officer with respect to medium shade in tea farming include ensuring that the tea plantations are developed and maintained up to the required standard of production and management. This includes monitoring the quantity and quality of tea produced, ensuring sufficient fertilizer application, controlling pests and diseases, and maintaining proper living conditions for employees. The regulatory officer also oversees the construction of roads and the diversification to other approved crops in areas unsuitable for tea cultivation. Additionally, they ensure that the operational activities of the tea industry comply with the required standards and that the tea is fit for human consumption.

CPU times: user 104 ms, sys: 4.06 ms, total: 108 ms
Wall time: 1.27 s


In [82]:
%%time

input_query = "When the parliament passed the Tea Commissioner's Act and how many times it got amended?"

result = query_engine.query(input_query)
display(Markdown(result.response))

The parliament passed the Tea Commissioner's Act on 27th January, 1983. It was amended three times.

CPU times: user 154 ms, sys: 1.11 ms, total: 155 ms
Wall time: 1.21 s


In [83]:
%%time

input_query = "List the all selling marks offered by SLTB for each Tea factory."

result = query_engine.query(input_query)
display(Markdown(result.response))

The context information provided does not mention specific selling marks offered by SLTB for each tea factory. It primarily discusses the Tea Control Act, No. 51 of 1957, and the Tea (Tax and Control of Export) Act, No. 16 of 1959, which focus on the regulation of tea manufacturing, export, and taxation.

CPU times: user 143 ms, sys: 1.03 ms, total: 144 ms
Wall time: 1.41 s


In [84]:
%%time

input_query = "List the all selling marks offered by Sri Lanka Transport Board for each Tea factory."

result = query_engine.query(input_query)
display(Markdown(result.response))

The context information provided does not mention specific selling marks offered by the Sri Lanka Transport Board for each Tea factory. The information primarily focuses on the regulations and standards related to the tea industry in Sri Lanka, including the Tea Control Act, No. 51 of 1957, the Sri Lanka Tea Board Law, No. 14 of 1975, and the Tea (Tax and Control of Export) Act, No. 16 of 1959.

CPU times: user 145 ms, sys: 1.97 ms, total: 147 ms
Wall time: 1.63 s


In [85]:
%%time

input_query = "Who can be a Green leaf dealer?"

result = query_engine.query(input_query)
display(Markdown(result.response))

A licensed dealer in green tea leaf, the registered proprietor of a registered estate or small holding, a registered manufacturer, or a person duly authorized in writing to act on behalf of such licensed dealer, registered proprietor, or registered manufacturer can be a green tea leaf dealer.

CPU times: user 146 ms, sys: 5.81 ms, total: 151 ms
Wall time: 1.44 s


In [88]:
%%time

input_query = "What is the annual registration fee for the Tea factory?"

result = query_engine.query(input_query)
display(Markdown(result.response))

The annual registration fee for a tea factory is not explicitly mentioned in the provided context. However, the context does mention various fees related to tea manufacturing and packaging. For instance, the Tea Control Act, No. 51 of 1957, mentions that the Controller decides whether a person is entitled to be registered as a manufacturer and whether a tea factory should be registered. It also mentions that the Controller may cancel the registration of any tea factory if the building, equipment, or manner of operation is not conducive to the manufacture of good-quality made tea.

The Gazette of the Democratic Socialist Republic of Sri Lanka EXTRAORDINARY, dated October 6, 2021, mentions a registration fee for tea packers. For large-scale local tea packers, the fee is Rs. 25,000; for medium-scale local tea packers, it is Rs. 10,000; for small-scale local tea packers, it is Rs. 5,000; and for export tea packers, it is Rs. 25,000.

The Sri Lanka Tea Board (Warehousing of Tea) Regulations, 1984, also mentions a registration fee of Rs. 25,000 for every registration made under these regulations.

However, none of these documents specifically mention an annual registration fee for a tea factory.

CPU times: user 128 ms, sys: 3.16 ms, total: 131 ms
Wall time: 1.8 s


In [89]:
%%time

input_query = "Under which clause of the act provides the Tea Commissioner to Reasonable price?"

result = query_engine.query(input_query)
display(Markdown(result.response))

The context information provided does not explicitly mention a clause that specifically deals with the Tea Commissioner providing a reasonable price. However, the Tea Commissioner is responsible for various duties and functions under the acts mentioned, such as controlling exports, inspecting premises, and appointing other officers and servants.

CPU times: user 131 ms, sys: 4.86 ms, total: 136 ms
Wall time: 1.13 s


In [90]:
%%time

input_query = "What are the main elevations and sub elevations of Tea growing area?"

result = query_engine.query(input_query)
display(Markdown(result.response))

The main elevations and sub-elevations of tea growing areas are not explicitly mentioned in the provided context. However, it is mentioned that low elevation tea plantations are more vulnerable to drought due to high ambient temperature during dry periods.

CPU times: user 136 ms, sys: 3.9 ms, total: 139 ms
Wall time: 1.08 s


### Delete tables after use<br>
**Do not run** if tables need to be reused later.

In [ ]:
# table.drop()